# IMPORTS

In [1]:
import os
from classes.layers.Conv2D import Conv2D
from classes.layers.Dense import Dense
from classes.layers.Flatten import Flatten
from classes.layers.Input import InputLayer
from classes.models.Sequential import Sequential
from classes.utils.ImageConvert import ImageConvert

# DATA PREPROCESSING

In [9]:
# TEST
## Constructor
import numpy as np
# c2d_layer_1 = Conv2D(1, (2, 2))
# c2d_layer_2 = Conv2D(
#     2, (4, 4), conv_stride=1, conv_padding_size=2,
#     activation='relu',
#     pool_kernel_size=9,
#     pool_stride=2,
#     pool_mode='avg',
#     name='c2d_l2')

# ## Class preview
# ### Conv kernels
# print(c2d_layer_1.conv_filters)
# print(c2d_layer_2.conv_filters)
## Compile
c2d_test = Conv2D(
    2, (2,2),
    conv_stride=1, conv_padding_size=0,
    activation='relu',
    pool_kernel_size=(2,2), pool_stride=1,
    pool_mode='max')
print("TYPE TEST:", type(c2d_test) is Conv2D)
c2d_test.compile((None, 6, 6, 5))
## Calculate
c2d_test.calculate(np.random.rand(6, 6, 5))
print('input')
print(c2d_test.input)
print(c2d_test.input.shape)
print()
print()
print('conv')
print(c2d_test.conv_output)
print(c2d_test.conv_output.shape)
print(c2d_test.conv_output[:,:,0])
print()
print()
print('filter')
print(c2d_test.conv_filters)
print(c2d_test.conv_filters[0])
print()
print()
print('output')
print(c2d_test.output)
print(c2d_test.output.shape)
# ## Detector
# test_detector = Conv2D(1, (2,2))
# test_detector_array = np.array([[[2, 7.2341], [-6,-0.1226]],
#             [[0,-0.1763],[-1,12.316872]]])
# print(test_detector_array)
# print()
# print()
# test_detector_result = test_detector.detect(test_detector_array)
# print(test_detector_result)
# print("TEST SHAPE :", test_detector_array.shape == test_detector_result.shape)
# ## Pooling
# test_pool_1 = Conv2D(
#     1, (2,2),
#     pool_kernel_size=(2,2),
#     pool_stride=1,
#     pool_mode='max',
# )
# test_pool_array = np.array([[[2, 7.2341], [-6, -0.1226]],
#             [[0, -0.1763],[-1, 12.316872]]])
# print(test_pool_array)
# print(test_pool_array.shape)
# test_pool_result = test_pool_1.pool(test_pool_array)
# print(test_pool_result)
# print(test_pool_result.shape)
c3d_test = Conv2D(
    2, (2,2),
    conv_stride=1, conv_padding_size=0,
    activation='relu',
    pool_kernel_size=(2,2), pool_stride=1,
    pool_mode='max')
print("TYPE TEST:", type(c2d_test) is Conv2D)
c3d_test.compile((None, 4, 4, 2))
## Calculate
c3d_test.calculate(np.random.rand(4, 4, 2))
print("----------")
c2d_test.backward(next_layer = c3d_test)
print(c2d_test.delta_pools)
print(c2d_test.delta_detectors)
print(c2d_test.delta_pools[0].shape)
print(c2d_test.delta_detectors[0].shape)
print('deltas')
print(c2d_test.deltas)

TYPE TEST: True
input
[[[0.19527372 0.58257153 0.12777738 0.26242842 0.88309583]
  [0.9115556  0.35662776 0.54444879 0.57962156 0.0843621 ]
  [0.62337379 0.27927584 0.81825508 0.52284149 0.45263561]
  [0.07214421 0.24909806 0.99137412 0.34869155 0.16089209]
  [0.19433136 0.89042622 0.2144942  0.50994209 0.04470644]
  [0.24627047 0.57005468 0.20820673 0.89979929 0.25640878]]

 [[0.37805697 0.86419522 0.69902197 0.6002127  0.02678939]
  [0.52527534 0.04647791 0.73439727 0.79017238 0.29161853]
  [0.43261028 0.58424465 0.90233843 0.70398141 0.92477814]
  [0.07009495 0.9676683  0.3389647  0.37328787 0.25023103]
  [0.37680481 0.46120113 0.4192405  0.04835354 0.83927883]
  [0.94107256 0.75930971 0.32901825 0.22814641 0.50250496]]

 [[0.80481166 0.56559273 0.75428016 0.43308974 0.82984135]
  [0.40056466 0.11011289 0.72447487 0.46513975 0.57329277]
  [0.79577584 0.33485999 0.17127355 0.15526926 0.82738558]
  [0.88144001 0.44496609 0.2993724  0.66169416 0.28114031]
  [0.92845325 0.87166464 0.391

In [2]:
ic_train = ImageConvert(
        rotate=30.,
        rescale=1./255.)
data_train_gen = ic_train.from_directory(os.path.join('.','data','train'), (256,256), mode='binary', color_mode='rgb')

ic_test = ImageConvert(
        rotate=30.,
        rescale=1./255.)
data_test_gen = ic_test.from_directory(os.path.join('.','data','test'), (256,256), mode='binary', color_mode='rgb')

In [3]:
X_train = []
y_train = []

for i in range(len(data_train_gen)):
    step = next(data_train_gen)
    X_train.append(step['data'])
    y_train.append(step['label'])

X_test = []
y_test = []

for i in range(len(data_test_gen)):
    step = next(data_test_gen)
    X_test.append(step['data'])
    y_test.append(step['label'])

# MODEL

## Instantiating

In [4]:
model = Sequential([
        InputLayer(input_shape=(256,256,3)),
        Conv2D(2, (16, 16), activation='relu'),
        Flatten(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

model.compile()

## Training

In [6]:
model.fit(X_train, y_train, 20, 10, 0.2, verbose=True)

TypeError: 'NoneType' object is not subscriptable

## Testing

In [7]:
# TEST
model.predict(X_test)

[array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.])]